In [1]:
from langfuse import Langfuse
from typing import List, Union
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_ollama import ChatOllama
from langfuse.callback import CallbackHandler
from langchain_core.messages import HumanMessage

In [2]:
ollama = ChatOllama(
    base_url="http://ollama:11434",
    model="llama3.2",
    temperature=0
)

In [3]:
langfuse_handler = CallbackHandler(host="http://langfuse-web:3000")

In [5]:
@tool
def addition(numbers: List[int]) -> int:
    """
    Sum all elements in the list.

    Args:
        numbers (List[int]): List of integers to sum.

    Returns:
        int: Sum of all integers in the list.

    Raises:
        ValueError: If the list of numbers is empty.
    """
    if not numbers:
        raise ValueError("The list of numbers cannot be empty")
    return sum(numbers)

@tool
def subtraction(numbers: List[int]) -> int:
    """
    Subtract sequentially all elements in the list (a - b - c - ...).

    Args:
        numbers (List[int]): List of integers where the first element is the minuend and the rest are subtrahends.

    Returns:
        int: Result of sequential subtraction.

    Raises:
        ValueError: If the list of numbers is empty.
    """
    if not numbers:
        raise ValueError("The list of numbers cannot be empty")
    result = numbers[0]
    for number in numbers[1:]:
        result -= number
    return result

@tool
def multiplication(numbers: List[int]) -> int:
    """
    Multiply all elements in the list.

    Args:
        numbers (List[int]): List of integers to multiply.

    Returns:
        int: Product of all integers in the list.

    Raises:
        ValueError: If the list of numbers is empty.
    """
    if not numbers:
        raise ValueError("The list of numbers cannot be empty")
    result = 1
    for number in numbers:
        result *= number
    return result

@tool
def division(numbers: List[int]) -> float:
    """
    Divide sequentially all elements in the list (a / b / c / ...).

    Args:
        numbers (List[int]): List of integers where the first element is the dividend and the rest are divisors.

    Returns:
        float: Result of sequential division.

    Raises:
        ValueError: If the list of numbers is empty.
        ZeroDivisionError: If any divisor is zero.
    """
    if not numbers:
        raise ValueError("The list of numbers cannot be empty")
    result = numbers[0]
    for number in numbers[1:]:
        if number == 0:
            raise ZeroDivisionError("Cannot divide by zero")
        result /= number
    return result

In [6]:
ollama_enhanced = ollama.bind_tools(my_tools:=[addition, subtraction, multiplication, division])
tool_node = ToolNode(my_tools)

In [13]:
tool_call = ollama_enhanced.invoke(
    [HumanMessage(content=f"I had 5 bananas, but now I only have 2 😔 How many have I eaten?")], 
    config={"callbacks": [langfuse_handler]}
)

In [14]:
response = tool_node.invoke({"messages": [tool_call]})

In [15]:
response

{'messages': [ToolMessage(content='3', name='subtraction', tool_call_id='77c2f469-7f59-44a6-91f9-7b2a10d0191c')]}